In [1]:
import psi4
import numpy as np
import matplotlib.pyplot as plt
import fortecubeview # Visualizar moléculas
import h5py
import pandas as pd
import CHE525_Vib as vib
%matplotlib inline

In [2]:
import pandas as pd
# Especificación del tamño de memoria que se utilizará en el cálculo
psi4.set_memory('5000 MB')
numpy_memory = 2 # Arreglos NumPy no pueden exceder 2 MB en tamaño

# archivo de salida
psi4.core.set_output_file('./iso_prop.dat', False)

# especificando la base
basis = 'cc-pvdz'

# Set computation options
#psi4.set_options({"PARALLEL": True,
#                 'reference': 'rhf'})

psi4.set_options({'basis': basis,
                  'scf_type': 'pk',
                  'e_convergence': 1e-8})

# ==> Definimos la Molécula con simetría inicial C1 <==
mol = psi4.geometry("""
 C        0.98463        0.00823       -0.04617
   C        2.43834       -0.21761        0.21335
   O        3.01834        0.55204        1.00571
   C        3.11922       -1.49787       -0.14575
   H        0.71052        0.98704        0.28804
   H        0.41005       -0.72318        0.48278
   H        0.79209       -0.07739       -1.09522
   H        2.66843        0.17259       -0.75603
   H        2.85554        0.26108        1.91661
   H        3.06636       -2.17424        0.68168
   H        4.14442       -1.30259       -0.38185
   H        2.63459       -1.93378       -0.99428
symmetry c1""")

# Optimizamos la molécula 
#psi4.energy('scf/cc-pvdz')
#psi4.core.clean()  #  Función para remover los scratch
scf_e, scf_wfn = psi4.frequency('scf/cc-pvdz', molecule=mol, return_wfn=True)
# Determinamos la energía y los modos vibracionales utilizando HF y cc-pvdz como base
#scf_e, scf_wfn = psi4.frequency('scf/cc-pvdz', molecule=mol, return_wfn=True)



  Memory set to   4.657 GiB by Python driver.


In [4]:
psi4.optimize('scf/cc-pvdz', molecule=mol)
psi4.core.clean()  #  Función para remover los scratch

Optimizer: Optimization complete!


In [8]:
scf_e, scf_wfn = psi4.frequency('scf/cc-pvdz', molecule=mol, return_wfn=True)
print(psi4.core.variables())

{'-D ENERGY': 0.0, 'CURRENT DIPOLE X': -0.9157448479700219, 'CURRENT DIPOLE Y': -1.317925879956788, 'CURRENT DIPOLE Z': -0.1740912726294754, 'CURRENT ENERGY': -193.1350144336746, 'CURRENT REFERENCE ENERGY': -193.1350144336746, 'ENTHALPY': -193.01416475986696, 'ENTHALPY CORRECTION': 0.12084967380764243, 'GIBBS FREE ENERGY': -193.04748300355575, 'GIBBS FREE ENERGY CORRECTION': 0.08753143011882977, 'HF TOTAL ENERGY': -193.1350144336746, 'NUCLEAR REPULSION ENERGY': 135.23170153468564, 'ONE-ELECTRON ENERGY': -530.0296402654408, 'PCM POLARIZATION ENERGY': 0.0, 'SCF DIPOLE X': -0.9157448479700219, 'SCF DIPOLE Y': -1.317925879956788, 'SCF DIPOLE Z': -0.1740912726294754, 'SCF ITERATION ENERGY': -193.1350144336746, 'SCF ITERATIONS': 11.0, 'SCF TOTAL ENERGY': -193.1350144336746, 'THERMAL ENERGY': -193.015108944415, 'THERMAL ENERGY CORRECTION': 0.11990548925959615, 'TWO-ELECTRON ENERGY': 201.66292429708056, 'ZERO K ENTHALPY': -193.02029794209795, 'ZPVE': 0.11471649157663517, 'CURRENT GRADIENT': <p

In [32]:
def wfn2molen_vibrational_modes(wfn):
    mol = wfn.molecule()
    geom = np.asarray(mol.geometry())
    atom_symbol = [mol.symbol(at) for at in range(mol.natom())]
    vibinfo = wfn.frequency_analysis
    """Format vibrational analysis for Molden.
    Parameters
    ----------
    vibinfo : dict of vibration Datum
        Holds results of vibrational analysis.
    atom_symbol : array-like of str
        (nat,) element symbols for geometry of vibrational analysis.
    geom : array-like of float
        (nat, 3) geometry of vibrational analysis [a0].
    standalone : bool, optional
        Whether returned string prefixed "[Molden Format]" for standalone rather than append.
    Returns
    -------
    str
        `vibinfo` formatted for Molden, including FREQ, FR-COORD, & FR-NORM-COORD fields.
    Notes
    -----
    Molden format spec from http://www.cmbi.ru.nl/molden/molden_format.html
    Specifies "atomic coordinates x,y,z and atomic displacements dx,dy,dz are all in Bohr (Atomic Unit of length)"
    Despite it being quite wrong, imaginary modes are represented by a negative frequency.
    
    Modified to be accepted by Avogrado. 
    """
    nat = int(len(vibinfo['q'].data[:, 0]) / 3)
    active = [idx for idx, trv in enumerate(vibinfo['TRV'].data) if trv == 'V']

    text = '[Molden Format]\n'

    text += """[FREQ]\n"""
    for vib in active:
        if vibinfo['omega'].data[vib].imag > vibinfo['omega'].data[vib].real:
            freq = -1.0 * vibinfo['omega'].data[vib].imag
            Int = vibinfo['IR_intensity'].data[vib].real
        else:
            freq = vibinfo['omega'].data[vib].real
            Int = vibinfo['IR_intensity'].data[vib].real
        text += """{:11.4f}{:11.4f}\n""".format(freq, Int)

        
    text += """[FR-COORD]\n"""
    for at in range(nat):
        text += ("{:>2}{:13.6f}{:13.6f}{:13.6f}\n").format(atom_symbol[at], *geom[at])

    text += """[FR-NORM-COORD]\n"""
    for idx, vib in enumerate(active):
        text += """vibration {}\n""".format(idx + 1)
        for at in range(nat):
            text += ('   ' + """{:11.6f}""" * 3 + '\n').format(*(vibinfo['x'].data[:, vib].reshape(nat, 3)[at].real))


    return text

In [33]:
# Writing result to ouput file
with open("iso_prop.molden", "w") as out:
    out.write(wfn2molen_vibrational_modes(scf_wfn))

In [23]:
scf_wfn.frequency_analysis['IR_intensity']

QCAspect(lbl='translation/rotation/vibration', units='', data=['TR', 'TR', 'TR', 'TR', 'TR', 'TR', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V'], comment='', doi=None, glossary='')

In [5]:
ques1 = input("Please input the name of PSI4 calculation output file: ")
readfile = open(ques1,'r')
sepfile = readfile.read().split('\n')
readfile.close()

freq = []
x = []
y = []
z = []
atom_name = []
atom_x = []
atom_y = []
atom_z = []
count = 0
count_ac = 0
count_s = 0
cc = 0
line = 0
for pair in sepfile:
    if pair == '*** PSI4 exiting successfully. Buy a developer a beer!':
        break
    else:
        pass

    if pair == '}':
        cc = 100000

    if line > 29 and cc < 100:
        matrix = pair.split()
        atom_name.append(matrix[0])
        atom_x.append(float(float(matrix[1])*1.889725989))
        atom_y.append(float(float(matrix[2])*1.889725989))
        atom_z.append(float(float(matrix[3])*1.889725989))

    if count_ac == 1 and pair != '':
        str_s = ''
        for i in range(13):
            str_s = str_s+pair[i]

        if count == 1:

            if count_s > (len(atom_name)-1):#19
                count = 0
                count_s = 0
            else:
                matrix = pair.split()
                count_s = count_s + 1
                x.append(float(matrix[1]))
                y.append(float(matrix[2]))
                z.append(float(matrix[3]))

        if str_s == '	     X      ':
            count = count + 1
        else:
            pass

        if str_s == '   Frequency:':
            c_f = ''
            for i in range(7):
                c_f = c_f+pair[20+i]
            if len(pair) >= 28:
                c_f = '-'+'%5f'%float(c_f)
            freq.append(c_f)

    if pair == '	Frequencies in cm^-1; force constants in au.':
        count_ac = count_ac + 1
    else:
        pass
    if line == 29:
        line = line + 1
    else:
        pass
    if pair == 'molecule {':
        line = 29
    else:
        pass
filename2 = input("Please input the name of the molden input file: ")
text_file1 = open(filename2,'w')
text_file1.write('[Molden Format]\n')
text_file1.write('[FR-COORD]\n')
for i in range(len(atom_name)):
    text_file1.write(str(atom_name[i]))
    text_file1.write('\t')
    text_file1.write(str(atom_x[i]))
    text_file1.write('\t')
    text_file1.write(str(atom_y[i]))
    text_file1.write('\t')
    text_file1.write(str(atom_z[i]))
    text_file1.write('\n')
text_file1.write('[FREQ]\n')
for i in range(len(freq)):
    text_file1.write(str(freq[i]))
    text_file1.write('\n')
text_file1.write('[FR-NORM-COORD]')
text_file1.write('\n')
aa = 0
for i in range(len(freq)):
    text_file1.write(str('Vibration'))
    text_file1.write('\t')
    text_file1.write(str(i+1))
    text_file1.write('\n')
    for j in range(len(atom_name)):
        text_file1.write('\t')
        text_file1.write(str(x[j+aa*len(atom_name)]*1.889725989))
        text_file1.write('\t')
        text_file1.write(str(y[j+aa*len(atom_name)]*1.889725989))
        text_file1.write('\t')
        text_file1.write(str(z[j+aa*len(atom_name)]*1.889725989))
        text_file1.write('\n')
    aa = aa + 1
text_file1.write('\n')
text_file1.close()

Please input the name of PSI4 calculation output file:  output-Copy1.dat
Please input the name of the molden input file:  out
